In [1]:
PDF_FILE = "IMM.pdf"
FILE = "address.pdf"

MODEL = "llama3.2"

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

second_loader = PyPDFLoader(FILE)
page_three = second_loader.load()

pages = pages + page_three

print(f"Number of pages: {len(pages)}")
print(f"Length of pages: {len(pages[1].page_content)}")
print("Content of pages:", pages[0].page_content)

Number of pages: 4
Length of pages: 2331
Content of pages: Providence St Vincent Medical Center 9205 SW Barnes Rd. Portland, OR 97225 (503) 216-1234 
 
Important Message from Medicare 
Patient name:     Patient number: 
 
Your Rights as a Hospital Inpatient: 
 
• You can receive Medicare covered services. This includes medically necessary hospital 
services and services you may need after you are discharged, if ordered by your doctor.  
You have a right to know about these services, who will pay for them, and where you can 
get them.  
 
• You can be involved in any decisions about your hospital stay. 
 
• You can report any concerns you have about the quality of care you receive to your QIO 
at: Acentra Health 1-888-305-6759 The QIO is the independent reviewer authorized by 
Medicare to review the decision to discharge you. 
 
• You can work with the hospital to prepare for your safe discharge and arrange for 
services you may need after you leave the hospital. When you no longer need

In [3]:
# Chunking
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print(f"Content of a chunk:", chunks[1].page_content)

Number of chunks: 9
Length of a chunk: 1206
Content of a chunk: the services you receive during the appeal (except for charges like copays and 
deductibles). 
 
• If you choose to appeal, the independent reviewer will ask for your opinion.  The 
reviewer also will look at your medical records and/or other relevant information.  You 
do not have to prepare anything in writing, but you have the right to do so if you wish.   
 
• If you choose to appeal, you and the reviewer will each receive a copy of a detailed 
explanation about why your covered hospital stay should not continue.  You will 
receive this detailed notice only after you request an appeal. 
 
• If the QIO finds that you are not ready to be discharged from the hospital, Medicare 
will continue to cover your hospital services.   
 
• If the QIO agrees services should no longer be covered after the discharge date, 
neither Medicare nor your Medicare health plan will pay for your hospital stay after 
noon of the day after the 

In [4]:
# Storing the chunks in vector store
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model=MODEL)
#vectorstore = FAISS.from_documents(chunks, embeddings)
import os

VECTOR_DB_PATH = "faiss_index"

print("Starting the vectorstore creation/loading process")

if os.path.exists(VECTOR_DB_PATH):
    print("Loading FAISS index from the disk")
    vectorstore = FAISS.load_local(VECTOR_DB_PATH, embeddings, allow_dangerous_deserialization=True)
    # Using allow_dangerous_deserialization=True is a security issue, be careful when uploading documents
else:
    print("Creating FAISS index and saving to the disk")
    chunks = splitter.split_documents(pages)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(VECTOR_DB_PATH)

Starting the vectorstore creation/loading process
Loading FAISS index from the disk


In [ ]:
# Setting up a retriever 
# We can use a retriever to find chunks in the vector store that are similar to a supplied question

retriever = vectorstore.as_retriever()
retriever.invoke("What if I miss the deadline?")

In [ ]:
# Configuring the Model
# We will use Ollama to load the local model in memory. After creating the model, we can invoke the model with a question and get the response back
from langchain_ollama import ChatOllama

model = ChatOllama(model=MODEL, temperature=0)
model.invoke("Who is the president of the United States?")

In [ ]:
# Parsing the model's response
# The response from the model is an AIMesssage instance containing the answer. We can extract the text answer by using the appropriate output parser.
# We can connect the model and a parser using a chain

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
print(chain.invoke("Who is the president of the United States?"))

In [ ]:
# Setting up a prompt
# In addition to the questions we want to ask, we also want to provide the model with the context of the PDF file. 
# We can use a prompt template to define and reuse the prompt we will use with the model

from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on a given context.

Answer the question based on the context. If you can't answer the question, reply "I don't know".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

In [ ]:
# Adding the prompt to the chain
# We can now chain the prompt with the model and the parser

chain = prompt | model | parser
chain.invoke(
    {"context": "If you have original Medicare call 1-888-305-6759",
     "question": "What number do ypu call if you have original medicare?"
    })
     

In [ ]:
# Adding the retriver to the chain
# Finally we can connect the retriever to the chain and get the context from the vector store

from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    #"What happens if I do not appeal?",
    #"What number do I call, if I have concerns about the quality of my care",
    #"Who do I call if I have providence medicare"
    #"Tell me Providence hospital address"
    #"What is Acentra Health"
    "Who do I report to with concerns about my quality of care",
    "Where is Providence St Vincent's Medical center in Portland Oregon and give me the address"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question':question})}")
    print("************************")



In [ ]:
# Notes